<a href="https://colab.research.google.com/github/springboardmentor54158a/AIDriven-Archaeological-Site-Mapping/blob/Shreyansh/AIDrivenArchaeologicalSiteMapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import os

BASE_PATH = '/content/drive/MyDrive/Ai Artifect Detection/data'

def create_multiclass_mask(img_path, ruins_mask_path, save_path):
    # Read image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Read ruins mask
    ruins = cv2.imread(ruins_mask_path, cv2.IMREAD_GRAYSCALE)
    ruins = cv2.resize(ruins, (img.shape[1], img.shape[0]))

    # Initialize final mask
    final_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)

    # ✅ Class 1: Ruins (ROBUST FIX)
    final_mask[ruins > 0] = 1

    # Class 2: Vegetation
    r, g, b = img[:,:,0], img[:,:,1], img[:,:,2]

    # NEW (robust vegetation detection)
    vegetation = (g > r + 10) & (g > b + 10) & (g > 80)


    final_mask[(vegetation) & (final_mask == 0)] = 2

    # Save (overwrite old mask)
    cv2.imwrite(save_path, final_mask)


# -------- RUN FOR ALL SPLITS --------
for split in ['train', 'val', 'test']:
    img_dir = os.path.join(BASE_PATH, split, 'images')
    mask_dir = os.path.join(BASE_PATH, split, 'masks')

    for fname in os.listdir(img_dir):
        img_path = os.path.join(img_dir, fname)
        ruins_mask_path = os.path.join(mask_dir, fname)

        create_multiclass_mask(img_path, ruins_mask_path, ruins_mask_path)

print("✅ Multiclass masks regenerated with ruins fix")


✅ Multiclass masks regenerated with ruins fix


In [3]:
import numpy as np
import cv2
import os

mask_dir = '/content/drive/MyDrive/Ai Artifect Detection/data/train/masks'
mask_name = os.listdir(mask_dir)[0]

test_mask = cv2.imread(
    os.path.join(mask_dir, mask_name),
    cv2.IMREAD_GRAYSCALE
)

np.unique(test_mask)


array([0, 1], dtype=uint8)

In [4]:
import torch
torch.cuda.is_available()


True

In [5]:
!pip install segmentation-models-pytorch albumentations --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.5 MB/s eta 0:00:00


In [6]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset

class ArchaeologyDataset(Dataset):
    def __init__(self, image_dir, mask_dir):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]

        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        image = torch.tensor(image, dtype=torch.float).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.long)

        return image, mask


In [7]:
from torch.utils.data import DataLoader

BASE_PATH = '/content/drive/MyDrive/Ai Artifect Detection/data'

train_dataset = ArchaeologyDataset(
    f"{BASE_PATH}/train/images",
    f"{BASE_PATH}/train/masks"
)

val_dataset = ArchaeologyDataset(
    f"{BASE_PATH}/val/images",
    f"{BASE_PATH}/val/masks"
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


In [8]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [9]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, masks in loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [10]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [11]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, masks in loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [12]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=1)


In [13]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset

class ArchaeologyDataset(Dataset):
    def __init__(self, image_dir, mask_dir):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]

        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # ✅ RESIZE HERE (SAFE)
        image = cv2.resize(image, (512, 512))
        mask  = cv2.resize(mask, (512, 512), interpolation=cv2.INTER_NEAREST)

        image = image / 255.0

        image = torch.tensor(image, dtype=torch.float).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.long)

        return image, mask


In [14]:
import torch
device = torch.device("cuda")
torch.cuda.empty_cache()
print(torch.cuda.get_device_name(0))



Tesla T4


In [15]:
class ArchaeologyDataset(Dataset):
    def __init__(self, image_dir, mask_dir):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]

        image = cv2.imread(os.path.join(self.image_dir, img_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))

        mask = cv2.imread(os.path.join(self.mask_dir, img_name), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)

        image = image / 255.0

        image = torch.tensor(image, dtype=torch.float).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.long)

        return image, mask


In [16]:
train_loader = DataLoader(
    train_dataset,
    batch_size=1,        # ❗ MUST be 1
    shuffle=True,
    pin_memory=True
)


In [17]:
model = smp.Unet(
    encoder_name="mobilenet_v2",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3
).to(device)


config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [18]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [19]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, masks in loader:
        images = images.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [20]:
EPOCHS = 10

for epoch in range(EPOCHS):
    loss = train_one_epoch(model, train_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {loss:.4f}")


Epoch [1/10] - Loss: 0.6735
Epoch [2/10] - Loss: 0.3790
Epoch [3/10] - Loss: 0.2804
Epoch [4/10] - Loss: 0.2139
Epoch [5/10] - Loss: 0.1746
Epoch [6/10] - Loss: 0.1504
Epoch [7/10] - Loss: 0.1343
Epoch [8/10] - Loss: 0.1300
Epoch [9/10] - Loss: 0.1186
Epoch [10/10] - Loss: 0.1079


In [21]:
def iou_score(pred, target, num_classes=3):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(1, num_classes):  # skip background
        pred_inds = pred == cls
        target_inds = target == cls

        intersection = (pred_inds & target_inds).sum().item()
        union = (pred_inds | target_inds).sum().item()

        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append(intersection / union)

    return sum([i for i in ious if i == i]) / len(ious)


def dice_score(pred, target, num_classes=3):
    dices = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(1, num_classes):
        pred_inds = pred == cls
        target_inds = target == cls

        intersection = (pred_inds & target_inds).sum().item()
        dice = (2 * intersection) / (pred_inds.sum() + target_inds.sum() + 1e-6)
        dices.append(dice)

    return sum(dices) / len(dices)


In [22]:
def validate(model, loader):
    model.eval()
    iou_total, dice_total = 0, 0

    with torch.no_grad():
        for images, masks in loader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            iou_total += iou_score(preds, masks)
            dice_total += dice_score(preds, masks)

    return iou_total / len(loader), dice_total / len(loader)


In [23]:
iou, dice = validate(model, val_loader)
print(f"IoU: {iou:.4f}, Dice: {dice:.4f}")


IoU: 0.1584, Dice: 0.2319


In [30]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17771, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 17771 (delta 95), reused 54 (delta 54), pack-reused 17616 (from 2)
Receiving objects: 100% (17771/17771), 17.13 MiB | 19.67 MiB/s, done.
Resolving deltas: 100% (12073/12073), done.
/content/yolov5/yolov5


In [31]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [29]:
%cd yolov5


[Errno 2] No such file or directory: 'yolov5'
/content/yolov5
